In [1]:
import scanpy as sc
import pandas as pd
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

In [2]:
target_variables_short = {
    "pearson_correlation_min_max": "pear_corr_min_max",
    "pearson_correlation_sigmoid": "pear_corr_sigmoid",
    "cosine_similarity_min_max": "cos_sim_min_max",
    "cosine_similarity_sigmoid": "cos_sim_sigmoid"
}

adata = sc.read("train_val_adata.h5ad")

adata_train, adata_val = adata[adata.obs["split"] == "train"], adata[adata.obs["split"] == "validation"]

predictions = {}
for target_var in target_variables_short.keys():
    target_var_short = target_variables_short[target_var]
    X_train, y_train = adata_train.X, adata_train.obs[target_var].values
    X_val, y_val = adata_val.X, adata_val.obs[target_var].values

    # Initialize and train the DummyRegressor
    model = DummyRegressor(strategy="mean")
    model.fit(X_train, y_train)

    y_val_pred = model.predict(X_val)

    # Store predictions for bootstrapping
    predictions[f"{target_var_short}-true"] = y_val
    predictions[f"{target_var_short}-pred"] = y_val_pred

    mse = mean_squared_error(y_val, y_val_pred)
    rmse = root_mean_squared_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)

    # Print results
    print(f"Validation {target_var_short} MSE: {mse:.4f}")
    print(f"Validation {target_var_short} RMSE: {rmse:.4f}")
    print(f"Validation {target_var_short} R2: {r2:.4f}\n")

Validation pear_corr_min_max MSE: 0.0056
Validation pear_corr_min_max RMSE: 0.0746
Validation pear_corr_min_max R2: -0.0003

Validation pear_corr_sigmoid MSE: 0.0060
Validation pear_corr_sigmoid RMSE: 0.0775
Validation pear_corr_sigmoid R2: -0.0001

Validation cos_sim_min_max MSE: 0.0089
Validation cos_sim_min_max RMSE: 0.0945
Validation cos_sim_min_max R2: -0.0002

Validation cos_sim_sigmoid MSE: 0.0092
Validation cos_sim_sigmoid RMSE: 0.0957
Validation cos_sim_sigmoid R2: -0.0000



In [3]:
pd.DataFrame(predictions).to_csv("baseline_mean.csv", index=False)